<a href="https://colab.research.google.com/github/owlbemi/capstone_test/blob/jk_test/capstone_testgc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install laspy
!pip install networkx
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
file_path = 'D:\Downloads\coffeeshop_merged.las'

In [ ]:
import laspy
import numpy as np
from scipy.spatial import KDTree
import networkx as nx

# Load .las file
with laspy.open(file_path) as f:
    for points in f.chunk_iterator(1_000_000):
        points = np.vstack((las.x, las.y, las.z)).T

In [ ]:
# Define start and end points
start_point = (x_start, y_start, z_start)
exit_point = (x_exit, y_exit, z_exit)

# Create KDTree for efficient distance calculations
kdtree = KDTree(points)

# Convert points to graph and find shortest path
G = nx.Graph()
for i, point in enumerate(points):
    # Link each point with its nearest neighbors (can adjust k as needed)
    distances, neighbors = kdtree.query(point, k=5)
    for dist, neighbor_idx in zip(distances, neighbors):
        if i != neighbor_idx:  # Avoid self-loop
            G.add_edge(i, neighbor_idx, weight=dist)

# Use Dijkstra's algorithm
path_indices = nx.dijkstra_path(G, source=start_index, target=exit_index)
path_points = points[path_indices]
